In [42]:
import numpy as np
import base64
from functools import cache

np.set_printoptions(formatter={'int':lambda i: f"{i:02x}".upper()}, linewidth=200)

In [43]:
def m(a, b):
    """Multiplication in GF(2^8)"""
    p = 0
    for _ in range(8):
        if b & 1 != 0:
            p = (p ^ a) & 255
        hb = (a & 0x80) != 0
        a = (a << 1) & 255
        if hb:
            a = (a ^ 0x1B) & 255
        b = (b >> 1) & 255
    return p

def mm(A, B):
    """Matrix Multiplication in GF(2^8)"""
    return np.array([[ np.bitwise_xor.reduce([m(A[i][k], B[k][j]) for k in range(A.shape[0])]) for j in range(B.shape[1])] for i in range(A.shape[0])], dtype=np.uint8)

def g(a):
    """Inverse in GF(2^8)"""
    if a == 0:
        return 0
    for i in range(256):
        if m(a, i) == 1:
            return i

In [44]:
rotl8 = lambda x, shift: ((x << shift) & 255) | ((x >> (8 - shift)) & 255)

f = lambda b: b ^ rotl8(b, 1) ^ rotl8(b, 2) ^ rotl8(b, 3) ^ rotl8(b, 4) ^ 0x63

invf = lambda s: rotl8(s, 1) ^ rotl8(s, 3) ^ rotl8(s, 6) ^ 0x5

In [45]:
sbox = np.array([[f(g(i << 4 | j)) for j in range(16)] for i in range(16)], dtype=np.uint8)
sbox

array([[63, 7C, 77, 7B, F2, 6B, 6F, C5, 30, 01, 67, 2B, FE, D7, AB, 76],
       [CA, 82, C9, 7D, FA, 59, 47, F0, AD, D4, A2, AF, 9C, A4, 72, C0],
       [B7, FD, 93, 26, 36, 3F, F7, CC, 34, A5, E5, F1, 71, D8, 31, 15],
       [04, C7, 23, C3, 18, 96, 05, 9A, 07, 12, 80, E2, EB, 27, B2, 75],
       [09, 83, 2C, 1A, 1B, 6E, 5A, A0, 52, 3B, D6, B3, 29, E3, 2F, 84],
       [53, D1, 00, ED, 20, FC, B1, 5B, 6A, CB, BE, 39, 4A, 4C, 58, CF],
       [D0, EF, AA, FB, 43, 4D, 33, 85, 45, F9, 02, 7F, 50, 3C, 9F, A8],
       [51, A3, 40, 8F, 92, 9D, 38, F5, BC, B6, DA, 21, 10, FF, F3, D2],
       [CD, 0C, 13, EC, 5F, 97, 44, 17, C4, A7, 7E, 3D, 64, 5D, 19, 73],
       [60, 81, 4F, DC, 22, 2A, 90, 88, 46, EE, B8, 14, DE, 5E, 0B, DB],
       [E0, 32, 3A, 0A, 49, 06, 24, 5C, C2, D3, AC, 62, 91, 95, E4, 79],
       [E7, C8, 37, 6D, 8D, D5, 4E, A9, 6C, 56, F4, EA, 65, 7A, AE, 08],
       [BA, 78, 25, 2E, 1C, A6, B4, C6, E8, DD, 74, 1F, 4B, BD, 8B, 8A],
       [70, 3E, B5, 66, 48, 03, F6, 0E, 61, 35, 57,

In [46]:
invsbox = np.array([[g(invf(i << 4 | j)) for j in range(16)] for i in range(16)], dtype=np.uint8)
invsbox

array([[52, 09, 6A, D5, 30, 36, A5, 38, BF, 40, A3, 9E, 81, F3, D7, FB],
       [7C, E3, 39, 82, 9B, 2F, FF, 87, 34, 8E, 43, 44, C4, DE, E9, CB],
       [54, 7B, 94, 32, A6, C2, 23, 3D, EE, 4C, 95, 0B, 42, FA, C3, 4E],
       [08, 2E, A1, 66, 28, D9, 24, B2, 76, 5B, A2, 49, 6D, 8B, D1, 25],
       [72, F8, F6, 64, 86, 68, 98, 16, D4, A4, 5C, CC, 5D, 65, B6, 92],
       [6C, 70, 48, 50, FD, ED, B9, DA, 5E, 15, 46, 57, A7, 8D, 9D, 84],
       [90, D8, AB, 00, 8C, BC, D3, 0A, F7, E4, 58, 05, B8, B3, 45, 06],
       [D0, 2C, 1E, 8F, CA, 3F, 0F, 02, C1, AF, BD, 03, 01, 13, 8A, 6B],
       [3A, 91, 11, 41, 4F, 67, DC, EA, 97, F2, CF, CE, F0, B4, E6, 73],
       [96, AC, 74, 22, E7, AD, 35, 85, E2, F9, 37, E8, 1C, 75, DF, 6E],
       [47, F1, 1A, 71, 1D, 29, C5, 89, 6F, B7, 62, 0E, AA, 18, BE, 1B],
       [FC, 56, 3E, 4B, C6, D2, 79, 20, 9A, DB, C0, FE, 78, CD, 5A, F4],
       [1F, DD, A8, 33, 88, 07, C7, 31, B1, 12, 10, 59, 27, 80, EC, 5F],
       [60, 51, 7F, A9, 19, B5, 4A, 0D, 2D, E5, 7A,

In [47]:
S = np.vectorize(lambda b: sbox[(b & 0xF0) >> 4][b & 0x0F])
iS = np.vectorize(lambda b: invsbox[(b & 0xF0) >> 4][b & 0x0F])

In [48]:
rc = np.array([0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1B, 0x10], dtype=np.uint8)
rcon = np.array([[rc[i], 0x00, 0x00, 0x00] for i in range(10)], dtype=np.uint8)

N = 4
K = np.frombuffer(b'Thats my Kung Fu', 'u1').reshape(-1, 4)
R = 11
RotWord = lambda b: np.array([b[1], b[2], b[3], b[0]])
SubWord = lambda b: np.array([S(b[0]), S(b[1]), S(b[2]), S(b[3])])

@cache
def W(i):
    if i < N:
        return K[i]
    else: # i >= N
        if i % N == 0:
            return W(i - N) ^ SubWord(RotWord(W(i - 1))) ^ rcon[i // N - 1]
        if N > 6 and i % N == 4:
            return W(i - N) ^ SubWord(W(i - 1))
        return W(i - N) ^ W(i - 1)

rkey = np.transpose(np.array([W(i) for i in range(4 * R)], dtype=np.uint8).reshape(-1, 4, 4), (0, 2, 1))

In [49]:
rkey[0]

array([[54, 73, 20, 67],
       [68, 20, 4B, 20],
       [61, 6D, 75, 46],
       [74, 79, 6E, 75]], dtype=uint8)

In [50]:
state = np.frombuffer(b'Two One Nine Two', 'u1').reshape(-1, 4).T
state

array([[54, 4F, 4E, 20],
       [77, 6E, 69, 54],
       [6F, 65, 6E, 77],
       [20, 20, 65, 6F]], dtype=uint8)

In [51]:
# AddRoundKey
state = state ^ rkey[0]
state

array([[00, 3C, 6E, 47],
       [1F, 4E, 22, 74],
       [0E, 08, 1B, 31],
       [54, 59, 0B, 1A]], dtype=uint8)

In [52]:
AddRoundKey = lambda state, key: state ^ key

In [53]:
# SubBytes
state = S(state)
state

array([[63, EB, 9F, A0],
       [C0, 2F, 93, 92],
       [AB, 30, AF, C7],
       [20, CB, 2B, A2]], dtype=uint8)

In [54]:
SubBytes = lambda state: S(state)
InvSubBytes = lambda state: iS(state)

In [55]:
# ShiftRows
state = np.array([np.roll(state[i], -i) for i in range(4)])
state

array([[63, EB, 9F, A0],
       [2F, 93, 92, C0],
       [AF, C7, AB, 30],
       [A2, 20, CB, 2B]], dtype=uint8)

In [56]:
ShiftRows = lambda state: np.array([np.roll(state[i], -i) for i in range(4)])

InvShiftRows = lambda state: np.array([np.roll(state[i], i) for i in range(4)])

In [57]:
# MixColumns

MDS = np.array([
    [2, 3, 1, 1],
    [1, 2, 3, 1],
    [1, 1, 2, 3],
    [3, 1, 1, 2]
])

state = mm(MDS, state)
state

array([[BA, 84, E8, 1B],
       [75, A4, 8D, 40],
       [F4, 8D, 06, 7D],
       [7A, 32, 0E, 5D]], dtype=uint8)

In [58]:
MixColumns = lambda state: mm(MDS, state)

invMDS = np.array([
    [14, 11, 13, 9],
    [9, 14, 11, 13],
    [13, 9, 14, 11],
    [11, 13, 9, 14]
])

InvMixColumns = lambda state: mm(invMDS, state)

In [59]:
state = state ^ rkey[1]
state

array([[58, 15, 59, CD],
       [47, B6, D4, 39],
       [08, 1C, E2, DF],
       [8B, BA, E8, CE]], dtype=uint8)

In [60]:
rc = np.array([0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1B, 0x36], dtype=np.uint8)
rcon = np.array([[rc[i], 0x00, 0x00, 0x00] for i in range(10)], dtype=np.uint8)

RotWord = lambda b: np.array([b[1], b[2], b[3], b[0]], dtype=np.uint8)
SubWord = lambda b: np.array([S(b[0]), S(b[1]), S(b[2]), S(b[3])], dtype=np.uint8)


def KeyExpansion(key):
    N = 4
    K = key.reshape(-1, 4)
    R = 11

    @cache
    def W(i):
        if i < N:
            return K[i]
        else: # i >= N
            if i % N == 0:
                return W(i - N) ^ SubWord(RotWord(W(i - 1))) ^ rcon[i // N - 1]
            if N > 6 and i % N == 4:
                return W(i - N) ^ SubWord(W(i - 1))
            return W(i - N) ^ W(i - 1)

    return np.transpose(
        np.array([
            W(i) for i in range(4 * R)
            ], dtype=np.uint8).reshape(-1, 4, 4),
        (0, 2, 1)
        )

In [61]:
def aes128_blk_enc(plain, key):

    plain = np.frombuffer(plain, 'u1')
    key = np.frombuffer(key, 'u1')

    rkey = KeyExpansion(key)

    state = plain.reshape(-1, 4).T

    state = AddRoundKey(state, rkey[0])

    for i in range(1, 10):
        state = SubBytes(state)
        state = ShiftRows(state)
        state = MixColumns(state)
        state = AddRoundKey(state, rkey[i])
    
    state = SubBytes(state)
    state = ShiftRows(state)
    state = AddRoundKey(state, rkey[10])

    return state.T.tobytes()

In [62]:
def aes128_blk_dec(cipher, key):

    cipher = np.frombuffer(cipher, 'u1')
    key = np.frombuffer(key, 'u1')

    rkey = KeyExpansion(key)

    state = cipher.reshape(-1, 4).T

    state = AddRoundKey(state, rkey[10])
    state = InvShiftRows(state)
    state = InvSubBytes(state)

    for i in reversed(range(1, 10)):
        state = AddRoundKey(state, rkey[i])
        state = InvMixColumns(state)
        state = InvShiftRows(state)
        state = InvSubBytes(state)
    
    state = AddRoundKey(state, rkey[0])

    return state.T.tobytes()

In [63]:
cipher = aes128_blk_enc(b"Two One Nine Two", b"Thats my Kung Fu")
np.frombuffer(cipher, 'u1')

array([29, C3, 50, 5F, 57, 14, 20, F6, 40, 22, 99, B3, 1A, 02, D7, 3A], dtype=uint8)

In [64]:
aes128_blk_dec(cipher, b"Thats my Kung Fu")

b'Two One Nine Two'

In [65]:
with open("YELLOW SUBMARINE.txt", "rb") as f:
    cipher = base64.decodebytes(f.read())

In [66]:
key = b'YELLOW SUBMARINE'

plain = b''.join(
    aes128_blk_dec(blk.tobytes(), key)
    for blk in np.frombuffer(cipher, 'u1').reshape(-1, 16)
    )

In [67]:
print(bytes.decode(plain))

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


In [68]:
def aes128_ecb_enc(plain, key):
    return b''.join(
        aes128_blk_enc(blk.tobytes(), key)
        for blk in np.frombuffer(plain, 'u1').reshape(-1, 16)
    )


def aes128_ecb_dec(cipher, key):
    return b''.join(
        aes128_blk_dec(blk.tobytes(), key)
        for blk in np.frombuffer(cipher, 'u1').reshape(-1, 16)
    )

In [69]:
aes128_ecb_dec(cipher, key)

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 